In [2]:
import calcbench as cb
import pandas as pd
import datetime
pit_columns = ['CIK', 'calendar_period', 'calendar_year', 'date_reported', 
               'fiscal_period', 'fiscal_year', 'metric', 'revision_number','ticker', 'value']

In [3]:
tickers = cb.companies(index='SP500').ticker

In [4]:
metrics = cb.available_metrics()
face_metrics = [m['metric'] for m in metrics['face']]

In [5]:
ticker_groups = [list(tickers[i:i + 100]) for i in range(0, len(tickers), 100)]

In [6]:
face_metrics = ['Assets']

In [7]:
data = pd.DataFrame()
elapsed_start = datetime.datetime.now()
for company_number, ticker_group in enumerate(ticker_groups):
    start = datetime.datetime.now()
    try:
        pit_data = cb.point_in_time(metrics=face_metrics, company_identifiers=list(ticker_group), all_history=True)
    except Exception as e:
        print("Exception getting {0}".format(ticker_group))
        raise e
    else:
        if not pit_data.empty:
            data = data.append(pit_data[pit_columns])
        continue
        print("done getting {0}, it took {1}, elapsed {2}, company_number {3}, average time {4}"\
              .format(ticker_group, 
                      datetime.datetime.now() - start,
                      datetime.datetime.now() - elapsed_start,
                      company_number,
                        (datetime.datetime.now() - elapsed_start) / (company_number + 1)))

In [8]:
missing_periods = []
tickers = data.ticker.unique()
#tickers = ['ALLE']
for ticker in tickers:
    test_data = data[(data.ticker == ticker) & (data.revision_number == 0)][['calendar_year', 'calendar_period', 'ticker']]
    test_dicts = [test_data.loc[i].to_dict() for i in test_data.index]
    last_row = test_dicts[0]
    for row in test_dicts[1:]:
        if last_row['calendar_period'] == 0:
            if row['calendar_period'] != 1:
                missing_periods.append(last_row)
        elif last_row['calendar_period'] == 4:
            if row['calendar_period'] != 0:
                missing_periods.append(last_row)
        elif last_row['calendar_period'] + 1 != row['calendar_period']:
            missing_periods.append(last_row)
        last_row = row

In [9]:
sorted(missing_periods, key=lambda row : row['calendar_year'], reverse=True)

[{'calendar_period': 2, 'calendar_year': 2016, 'ticker': 'A'},
 {'calendar_period': 3, 'calendar_year': 2016, 'ticker': 'ADBE'},
 {'calendar_period': 2, 'calendar_year': 2016, 'ticker': 'AZO'},
 {'calendar_period': 3, 'calendar_year': 2016, 'ticker': 'CCL'},
 {'calendar_period': 2, 'calendar_year': 2016, 'ticker': 'COST'},
 {'calendar_period': 4, 'calendar_year': 2016, 'ticker': 'FDX'},
 {'calendar_period': 4, 'calendar_year': 2016, 'ticker': 'GIS'},
 {'calendar_period': 2, 'calendar_year': 2016, 'ticker': 'HPE'},
 {'calendar_period': 2, 'calendar_year': 2016, 'ticker': 'HPQ'},
 {'calendar_period': 3, 'calendar_year': 2016, 'ticker': 'MAA'},
 {'calendar_period': 4, 'calendar_year': 2016, 'ticker': 'ORCL'},
 {'calendar_period': 3, 'calendar_year': 2016, 'ticker': 'REG'},
 {'calendar_period': 0, 'calendar_year': 2015, 'ticker': 'AZO'},
 {'calendar_period': 3, 'calendar_year': 2015, 'ticker': 'COO'},
 {'calendar_period': 0, 'calendar_year': 2015, 'ticker': 'COO'},
 {'calendar_period': 0, 

In [11]:
pd.DataFrame(missing_periods).groupby(['calendar_year']).agg(['count'])

,calendar_period,ticker
,count,count
calendar_year,,
2007,14,14
2008,297,297
2009,127,127
2010,24,24
2011,12,12
2012,12,12
2013,6,6
2014,17,17


In [16]:
groups = data.groupby('ticker')

In [17]:
groups

In [42]:
list(groups.groups.items())[0]

('CSRA',
 Int64Index([835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846], dtype='int64'))

In [44]:
data.iloc(list(groups.groups.items())[0][1])

In [50]:
rows = data.iloc[list(groups.groups.items())[0][1]]

In [56]:
data.iloc[0]['calendar_period']

3

In [62]:
list(groups.groups.items())[0][1][1:]

Int64Index([836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846], dtype='int64')